In [1]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from sklearn.utils import shuffle
from sklearn.utils import check_random_state
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.metrics import pairwise_distances                 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import make_blobs
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.neighbors import NearestNeighbors, DistanceMetric
from sklearn.preprocessing import normalize

from scipy.sparse import csr_matrix 

import pandas as pd
import numpy as np
import sys      
import os
%matplotlib inline

# Load user dataset

In [86]:
# import text documents from user dataset
user_data=pd.read_csv('../../preprocess_data/preprocessed/user_appRuns_long_2020_05_01.csv')
user_data = user_data.set_index(['username'])

In [85]:
# import text documents from user information
user_info=pd.read_csv('../../preprocess_data/preprocessed/user_info_dataset_2020_05_01.csv')

In [59]:
# extract user information
dataset_uinfo= user_info[['username', 'occupation_id', 'research_area_id']]
dataset_uinfo=dataset_uinfo.drop_duplicates()
dataset_uinfo = dataset_uinfo.set_index(['username'])
dataset_uinfo.shape

(83230, 2)

# compute K-NN between users

In [26]:
# normalize dataset
#user_data_norm= normalize(user_data, norm='l2')

In [81]:
nbrs = NearestNeighbors(n_neighbors=10, metric='cosine').fit(user_data)
distances, indices = nbrs.kneighbors()

def get_closest_neighs(app_id):
    row = user_data.index.get_loc(app_id)
    usernames=user_data.iloc[indices[row],].reset_index()['username'].values
    result = pd.DataFrame({'username':usernames,
                           'domain': dataset_uinfo.iloc[dataset_uinfo.index.get_indexer(usernames)]['research_area_id'],
                           'occupation': dataset_uinfo.iloc[dataset_uinfo.index.get_indexer(usernames)]['occupation_id'],
                           'distance': distances[row].flatten()})
    return result

## Test the K-NN implementation

In [82]:
test_user="2016154805"

In [87]:
users=get_closest_neighs(test_user)

In [84]:
# get info about query user
dataset_uinfo.loc[test_user]

occupation_id        3
research_area_id    65
Name: 2016154805, dtype: int64